# Sample commands to setup Metabase

2019-04-09

Places for improvement are marked with `TODO`

The Metabase project is designed to track metadata and ETL processes automatically for logging and validation purposes. Its current ERD can be found at TODO.

## Requirements

### PostgreSQL

The project is developed with PostgreSQL 9.5, which can be downloaded from https://www.postgresql.org/download/.

### Python

The project is built on Python 3.5. Below are some sample commands to create a virtual environment for this project with [Anaconda](https://www.anaconda.com/distribution/) and load prerequisites into that environment.

In [4]:
!conda create -y -n adrf35 python=3.5

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\shan\AppData\Local\Continuum\anaconda3\envs\adrf35

  added / updated specs:
    - python=3.5


The following NEW packages will be INSTALLED:

  certifi            pkgs/main/win-64::certifi-2018.8.24-py35_1
  pip                pkgs/main/win-64::pip-10.0.1-py35_0
  python             pkgs/main/win-64::python-3.5.6-he025d50_0
  setuptools         pkgs/main/win-64::setuptools-40.2.0-py35_0
  vc                 pkgs/main/win-64::vc-14.1-h0510ff6_4
  vs2015_runtime     pkgs/main/win-64::vs2015_runtime-14.15.26706-h3a45250_0
  wheel              pkgs/main/win-64::wheel-0.31.1-py35_0
  wincertstore       pkgs/main/win-64::wincertstore-0.2-py35hfebbdb8_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use:
# > activate adrf35
#
# To deactivate an active environment, use:
# > deactivate
#


The `-y` flag confirms the creation; the `-n` flag expects a name for the newly created environment. In this case, it is called "adrf35." The warning message at the bottom can be ignored since it is a [open issue](https://github.com/conda/conda/issues/8512) of Anaconda 4.6.11 that has not been fixed as of April 9, 2019. 

To activate the virtual environment we just created and install required packages into it, run the following commands in terminal:

```
conda activate adrf35

pip install -r requirements.txt
```

Since shell commands prefixed with `!` in Jupyter Notebooks are executed in a temporary subshell with the default virtual environment, it would be a lot easier to run the above commands in an actual terminal.

## Preparing the database

Metabase writes metadata to a `metabase` schema as superuser `metaadmin`. These can be configured in [`metabase/settings.py`](metabase/settings.py). By default, we need to first create a superuser with login privilege and store its database credentials in a [pgpass](https://www.postgresql.org/docs/9.5/libpq-pgpass.html) file.

The sample codes below create superuser `metaadmin` and schema `metabase` with sqlalchemy.

In [3]:
import sqlalchemy

In [4]:
engine = sqlalchemy.create_engine('postgres://postgres@localhost/postgres')

In [ ]:
engine.execute("""
    CREATE ROLE metaadmin WITH LOGIN SUPERUSER;
    SET ROLE metaadmin;
    CREATE SCHEMA metabase;
""").close()

To initiate metabase tables under the `metabase` schema, run an [Alembic](https://alembic.sqlalchemy.org/en/latest/) migration with the following command:

In [1]:
!alembic upgrade head

It runs the migration scripts under the [`alembic`](alembic/) folder.

Now the `metabase` schema and its tables are ready to host metadata.

## TODO: Command line interface / JSON config usage

## For developers

### Lint

We use [flake8](http://flake8.pycqa.org/en/latest/) for style guide enforcement. Flake8 can be installed with the following command:

```
pip install flake8
```

and run the linter over the project directory with

In [3]:
!flake8 .

Note that, as mentioned above, `!`shell commands are executed in temporary subshells with default virtual environments. The above command works only if  `flake8` is installed in your default environment (`base`). If you just want to install `flake8` in the `adrf35` environment and call it from there in Jupyter Notebook, you can try the following workaround:

In [22]:
%%script cmd

conda activate adrf35
flake8 .

Microsoft Windows [Version 6.1.7601]
Copyright (c) 2009 Microsoft Corporation.  All rights reserved.

C:\Users\shan\Desktop\git\adrf-metabase>conda activate adrf35

(adrf35) C:\Users\shan\Desktop\git\adrf-metabase>flake8 .

(adrf35) C:\Users\shan\Desktop\git\adrf-metabase>

In [23]:
%killbgscripts

All background processes were killed.


It uses the [%%script](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-script) cell magic to activate `adrf35` and run `flake8` by `cmd` on Windows (or `sh` or `bash` on Linux). The `%killbgscripts` line magic kills that script  upon completion. Similarly hereafter.

### Tests

We use [pytest](https://doc.pytest.org/) for unit test and [testing.postgresql](https://github.com/tk0miya/testing.postgresql) to setup testing databases. The former can be installed with

```
pip install pytest
```

Note that the testing.postgresql 1.3.0 on PyPI has an [open issue](https://github.com/tk0miya/testing.postgresql/issues/16) that can lead to false errors on Windows systems. It can be avoided by installing their master branch on GitHub via

```
pip install git+https://github.com/tk0miya/testing.postgresql.git
```

As of April 9, 2019, its PyPI distribution works for Linux, but Linux users may also want to install from the master branch since it seems that that project is no longer active.

A sample printout from a test session may look like below:

In [5]:
!pytest tests/

============================= test session starts =============================
platform win32 -- Python 3.7.1, pytest-4.0.2, py-1.7.0, pluggy-0.8.0
rootdir: C:\Users\shan\Desktop\git\adrf-metabase, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.2.0, arraydiff-0.3
collected 6 items

tests\test_extract_metadata.py ......                                    [100%]

========================== 6 passed in 56.99 seconds ==========================


### Test coverage

We use [pytest-cov](https://pytest-cov.readthedocs.io/en/latest/) as a plugin for pytest to ensure test coverage. It can be installed with 

```
pip install pytest-cov
```

and run like

In [6]:
!pytest --cov=metabase tests --cov-report html

============================= test session starts =============================
platform win32 -- Python 3.7.1, pytest-4.0.2, py-1.7.0, pluggy-0.8.0
rootdir: C:\Users\shan\Desktop\git\adrf-metabase, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.2.0, cov-2.6.1, arraydiff-0.3
collected 6 items

tests\test_extract_metadata.py ......                                    [100%]

----------- coverage: platform win32, python 3.7.1-final-0 -----------
Coverage HTML written to dir htmlcov


========================== 6 passed in 66.91 seconds ==========================


### Documentation

Documentation of this project is built with [Sphinx](http://www.sphinx-doc.org/en/master/), which can be installed with 

```
pip install sphinx
```

Also, an online build of the documentation is hosted by [Read the Docs](https://readthedocs.org/) and can be found at https://adrf-metabase.readthedocs.io.

To build the documentation locally, first go to the [`docs/`](docs/) folder

In [ ]:
cd docs/

and run [`sphinx-apidoc`](https://www.sphinx-doc.org/en/master/man/sphinx-apidoc.html) to generate/update `rst` files under [`docs/source/`](docs/source/).

In [8]:
!sphinx-apidoc -o source/ ../metabase --force --separate

Creating file source/metabase.extract_metadata.rst.
Creating file source/metabase.extract_metadata_helper.rst.
Creating file source/metabase.settings.rst.
Creating file source/metabase.rst.
Creating file source/modules.rst.


In the sample command above, `-o source/` specifies the output directory as `source/`; `../metabase` is our module path; `--force` overwrites existing `rst` files; `--separate` puts documentation for each module on its own page.

Last, documentation can be rendered as HTML with

In [ ]:
!make html

or PDF with

In [ ]:
!make latexpdf

Note that `latexpdf` has some prerequisites that may take some time (> 30 minutes) and space (several GBs) to install. Information about the dependencies can be found on the [LaTexBuilder documentation](http://www.sphinx-doc.org/en/master/usage/builders/index.html#sphinx.builders.latex.LaTeXBuilder) for Linux and [TeX Live](https://tug.org/texlive/windows.html) for Windows.

The outputs can be found under [`docs/build/`](`docs/build/`).